In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

In [ ]:
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import skew, kurtosis, boxcox_normmax
from scipy.special import boxcox1p

from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor

# Establish File System location
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Note from Kaggle
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessiond

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Mute warnings
warnings.filterwarnings('ignore')

In [ ]:
# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p, boxcox
from scipy.stats import boxcox_normmax,yeojohnson

# Misc
import optuna
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# Interpretability 
from pdpbox import pdp, get_dataset, info_plots
import shap
import eli5
from eli5.sklearn import PermutationImportance

--- 
---
# Outline for Refactoring Code

## load Raw Data
## Preprocessing
    * impute
    * clean
    * encode

## Feature Engineering
    * optional normalization

## Train Individual Models
    * Linear Based
    * Tree-like
    * XGboost-like
    * Stacked
    
## Blended Model
    * Is there a way to cluster
    
## Machine Interpretability 
    * final blended model
    * most effective individual models

## Presentation
    * possible application of interpreted models
    
---
---

In [ ]:
def load_data():
    # Read data
    data_dir = Path("../input/house-prices-advanced-regression-techniques/")
    df_train = pd.read_csv(data_dir / "train.csv", index_col="Id")
    df_test = pd.read_csv(data_dir / "test.csv", index_col="Id")
    # Merge the splits so we can process them together
    df = pd.concat([df_train, df_test])
    # Preprocessing
    df = clean(df)
    df = encode(df)
    df = impute_simple(df)
    # Reform splits
    df_train = df.loc[df_train.index, :]
    df_test = df.loc[df_test.index, :]
    return df_train, df_test


def load_data_combined():
    # Read data
    #../input/house-prices-advanced-regression-techniques/test.csv
    data_dir = Path("../input/house-prices-advanced-regression-techniques/")
    df_train = pd.read_csv(data_dir / "train.csv", index_col="Id")
    df_test = pd.read_csv(data_dir / "test.csv", index_col="Id")
    # Merge the splits so we can process them together
    return pd.concat([df_train, df_test])

def load_data_simple(): 
    # Read data
    #../input/house-prices-advanced-regression-techniques/test.csv
    data_dir = Path("../input/house-prices-advanced-regression-techniques/")
    df_train = pd.read_csv(data_dir / "train.csv", index_col="Id")
    df_test = pd.read_csv(data_dir / "test.csv", index_col="Id")
    # Merge the splits so we can process them together
    return df_train, df_test

def reform_train_test_split(df_combined):
    # Read data
    #../input/house-prices-advanced-regression-techniques/test.csv
    data_dir = Path("../input/house-prices-advanced-regression-techniques/")
    df_train = pd.read_csv(data_dir / "train.csv", index_col="Id")
    df_test = pd.read_csv(data_dir / "test.csv", index_col="Id")
    # Merge the splits so we can process them together
    df_train = df_combined.loc[df_train.index, :]
    df_test = df_combined.loc[df_test.index, :]
    return df_train, df_test

In [ ]:
def clean(df):
    df["Exterior2nd"] = df["Exterior2nd"].replace({"Brk Cmn": "BrkComm"})
    # Some values of GarageYrBlt are corrupt, so we'll replace them
    # with the year the house was built
    df["GarageYrBlt"] = df["GarageYrBlt"].where(df.GarageYrBlt <= 2010, df.YearBuilt)
    # Names beginning with numbers are awkward to work with
    df.rename(columns={
        "1stFlrSF": "FirstFlrSF",
        "2ndFlrSF": "SecondFlrSF",
        "3SsnPorch": "Threeseasonporch",
    }, inplace=True,
    )
    return df

In [ ]:
def impute_simple(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("category"):
        df[name] = df[name].fillna("None")
    return df

In [ ]:
# -----------------------
# Normalize Numerical Variables that do not originally have a gaussian (normal) distribution

def normalize_numerics(df, skew_cutoff=0.5):
    '''
    Finds Features with large skewness and applies box-cox transformation
    '''
    X = df.copy()
    numeric_columns = X.select_dtypes(["number"]).columns
    skew_feats = X[numeric_columns].apply(lambda x: skew(x)).sort_values(ascending=False)
    
    too_skew = skew_feats[skew_feats > skew_cutoff].index
    
    # normalize each of the features with high skew with scipy boxcox 
    for s in too_skew:
        X[s] = boxcox1p(X[s], boxcox_normmax(X[s] + 1))
    return X

In [ ]:
def find_skew_cols(df):
    X = df.copy()
    numeric_columns = X.select_dtypes(["number"]).columns
    skew_feats = X[numeric_columns].apply(lambda x: skew(x)).sort_values(ascending=False)
    return skew_feats

In [ ]:
def de_skew_yeo(df):
    skews = find_skew_cols(df)
    skew_check = skews[skews > 0.5].index.tolist()
    
    for s in skew_check: 
        df[s] = yeojohnson(df[s])[0]
    return df

In [ ]:
# label encoding for tree-based models
def label_encode_manual(df):
    '''tested: passing'''
    X = df.copy()
    for colname in X.select_dtypes(["category"]):
        X[colname] =  X[colname].cat.codes
        
    return X

# label encoding using sklearn module LabelEncoder
def label_encode_sk(df):
    '''tested: passing'''
    X = df.copy()
    categorical_columns = X.select_dtypes(["category"]).columns
    
    le = LabelEncoder()
    X[categorical_columns] = X[categorical_columns].apply(lambda x: le.fit_transform(x))
    return X

# One Hot encoding for linear regression based models, and svm
def one_hot_encode_pd(df):
    '''tested: passing'''
    X = df.copy()
    # for now we just use all categorical variables
    cols_for_one_hot = X.select_dtypes(["category"]).columns.tolist()
        
    X_ohe = pd.get_dummies(X, columns=cols_for_one_hot, prefix=cols_for_one_hot, drop_first=True)
    #X = X.drop(cols_for_one_hot)
    #X = pd.concat((X,X_ohe), axis=1)
    return X_ohe 
        
def one_hot_encode_sk(df):
    '''tested: passing'''
    X = df.copy()
    cols_for_one_hot = X.select_dtypes(["category"]).columns
    
    ohe = OneHotEncoder()
    X[cols_for_one_hot] = X[cols_for_one_hot].apply(lambda x: ohe.fit_transform(x))
    
    return X

def apply_standard_scaler(df):
    X = df.copy

In [ ]:
#-------------------------------
# Encode

# The numeric features are already encoded correctly (`float` for
# continuous, `int` for discrete), but the categoricals we'll need to
# do ourselves. Note in particular, that the `MSSubClass` feature is
# read as an `int` type, but is actually a (nominative) categorical.

# The nominative (unordered) categorical features
features_nom = ["MSSubClass", "MSZoning", "Alley", "LandContour", "LotConfig", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating", "CentralAir", "GarageType", "MiscFeature", "SaleType", "SaleCondition"]


# The ordinal (ordered) categorical features 

# Pandas calls the categories "levels"
five_levels = ["Po", "Fa", "TA", "Gd", "Ex"]
ten_levels = list(range(10))

ordered_levels = {
    "OverallQual": ten_levels,
    "OverallCond": ten_levels,
    "ExterQual": five_levels,
    "ExterCond": five_levels,
    "BsmtQual": five_levels,
    "BsmtCond": five_levels,
    "HeatingQC": five_levels,
    "KitchenQual": five_levels,
    "FireplaceQu": five_levels,
    "GarageQual": five_levels,
    "GarageCond": five_levels,
    "PoolQC": five_levels,
    "LotShape": ["Reg", "IR1", "IR2", "IR3"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "BsmtExposure": ["No", "Mn", "Av", "Gd"],
    "BsmtFinType1": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "BsmtFinType2": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "Functional": ["Sal", "Sev", "Maj1", "Maj2", "Mod", "Min2", "Min1", "Typ"],
    "GarageFinish": ["Unf", "RFn", "Fin"],
    "PavedDrive": ["N", "P", "Y"],
    "Utilities": ["NoSeWa", "NoSewr", "AllPub"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"],
    "Fence": ["MnWw", "GdWo", "MnPrv", "GdPrv"],
}

# Add a None level for missing values
# DH comment - this concatenates none - the pattern is simple and clever remember for the future
ordered_levels = {key: ["None"] + value for key, value in
                  ordered_levels.items()}

features_ord = [key for key in ordered_levels.keys()]
features_cat = features_ord + features_nom

# DH Comment = the df[].cat.*     functions below are new. remember their usefulness
def encode(df):
    # Nominal categories
    for name in features_nom:
        df[name] = df[name].astype("category")
        # Add a None category for missing values
        if "None" not in df[name].cat.categories:
            df[name].cat.add_categories("None", inplace=True)
    # Ordinal categories
    # DH comment - how to apply the Ordinal Ordering
    for name, levels in ordered_levels.items():
        df[name] = df[name].astype(CategoricalDtype(levels,
                                                  ordered=True))
    return df


In [ ]:
# --------------------------
# Data Processing Utility Functions
def preprocess_data(df):
    df = clean(df)
    df = encode(df)
    df = impute_simple(df)
    return df
        
def load_and_preproc_data(type='basic'): 
    if type=='basic':
        df_1 = load_data_combined()
        df_1 = preprocess_data(df_1)
        df_train, df_test = reform_train_test_split(df_1)
        return df_train, df_test
    
    if type=='tt_split':
        df, _ = load_data()
        df = preprocess_data(df)
        
        df = clean(df)
        df = encode(df)
        df = impute_simple(df)
        
        y = df['SalePrice']
        df = df.drop(['SalePrice'], axis=1)
        
        X_tn, X_tt, y_tn, y_tt = train_test_split(df, y, test_size=0.2, random_state=42)
        return X_tn, X_tt, y_tn, y_tt
    #active
    else:
        raise ValueError



---
---
# Feature Engineering

In [ ]:

def score_dataset(X, y, model=XGBRegressor(), enc=True, trans_targ=True):
    # Label encoding for categoricals
    #
    # Label encoding is good for XGBoost and RandomForest, but one-hot
    # would be better for models like Lasso or Ridge. The `cat.codes`
    # attribute holds the category levels.
    if enc:
        for colname in X.select_dtypes(["category"]):
            X[colname] = X[colname].cat.codes
    
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    y = np.log(y) if trans_targ else y
    
    score = cross_val_score(
            model, X, y, cv=5, scoring="neg_mean_squared_error"
        )
    
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

In [ ]:
# -----------------------------------------
# Baseline Scoring: for later comparison 
def get_baseline_xgb_score():
    df_train, df_test = load_and_preproc_data()

    X = df_train.copy()
    y = X.pop("SalePrice")

    baseline_score = score_dataset(X, y)
    print(f"Baseline XGB score: {baseline_score:.5f} RMSLE")

In [ ]:
get_baseline_xgb_score()

In [ ]:
#-------------------------
# Mutual Information

def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    
#look at the feature scores
# X = df_train.copy()
# y = X.pop("SalePrice")

# mi_scores = make_mi_scores(X, y)
# mi_scores

In [ ]:
def drop_uninformative(df, mi_scores):
    return df.loc[:, mi_scores > 0.0]

In [ ]:
def test_drop_uninformative():
    X_train = df_train.copy()
    y_train = X_train.pop("SalePrice")
    X_train = drop_uninformative(X_train, mi_scores)

# score_dataset(X_train, y_train)

# Creating Our Own Features

In [ ]:
# -----------------------
# Mathematical Transforms

def mathematical_transforms(df):
    X = pd.DataFrame()
    X["LivLotRatio"] = df.GrLivArea / df.LotArea
    X["Spaciousness"] = (df.FirstFlrSF + df.SecondFlrSF) / df.TotRmsAbvGrd
    return X
    
def interactions(df):
    X = pd.get_dummies(df.BldgType, prefix="Bldg")
    X = X.mul(df.GrLivArea, axis=0)
    return X

def porch_counts(df):
    X = pd.DataFrame()
    X["PorchTypes"] = df[[
        "WoodDeckSF",
        "OpenPorchSF",
        "EnclosedPorch",
        "Threeseasonporch",
        "ScreenPorch",
    ]].gt(0.0).sum(axis='columns')
    return X

def class_break_down(df):
    X = pd.DataFrame()
    X["MSClass"] = df.MSSubClass.str.split("_",n=1, expand=True)[0]
    return X

def group_transforms(df):
    X = pd.DataFrame()
    X["MedNgbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")
    return X

In [ ]:
# --------------------------------
# Clustering Features

cluster_features = [
    "LotArea",
    "TotalBsmtSF",
    "FirstFlrSF",
    "SecondFlrSF",
    "GrLivArea",
]

def cluster_labels(df, features, n_clusters):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
    X_new = pd.DataFrame()
    X_new["Cluster"] = kmeans.fit_predict(X_scaled)
    return X_new

def cluster_distance(df, features, n_clusters=20): 
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0))/ X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=20, n_init=50, random_state=0)
    X_clustered = kmeans.fit_transform(X_scaled)
    X_clustered = pd.DataFrame(
        X_clustered, columns=[f"Centroid_{i}" for i in range(X_clustered.shape[1])]
    )
    return X_clustered
    

In [ ]:
#-------------------------------
# Principal Component Analysis

#--------------------------------
# Utility Functions for creating the principal components
def apply_pca(X, standardize=True):
    # standardize the original features
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create Principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T, # transpose the matrix of loadings
        columns=component_names, # the columns are the principal components
        index=X.columns, # and the rows are the original features
    )
    return pca, X_pca, loadings

def plot_variance(pcs, width=8, dpi=100):
        # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

In [ ]:
def get_pca_feature_names():
    pca_features = [
        "GarageArea",
        "YearRemodAdd",
        "TotalBsmtSF",
        "GrLivArea",
    ]
    return pca_features

def pca_driven_features(df):
    X = pd.DataFrame()
    X["GrLiv+TotalBsmtSF"] = df.GrLivArea + df.TotalBsmtSF
    X["YrRemod*TotalBsmtSF"] = df.YearRemodAdd * df.TotalBsmtSF
    return X

def pca_components(df, features):
    X = df.loc[:, features]
    t0, X_pca, t_1 = apply_pca(X)
    return X_pca

In [ ]:
def indicate_outliers(df):
    X_new = pd.DataFrame()
    X_new["Outlier"] = (df.Neighborhood == "Edwards") & (df.SaleCondition == "Partial")
    
    # outlier masks
    grliv_mask = (df['GrLivArea'] > 4000) & (X_new["Outlier"] == False)
    
    # apply masks
    X_new.loc[grliv_mask, "Outlier"] = True
    
    # convert bool to int
    X_new["Outlier"] = np.where(X_new["Outlier"]==True, 1, 0)
    return X_new

In [ ]:
def corrplot(df, method="pearson", annot=True, **kwargs):
    sns.clustermap(
        df.corr(method),
        vmin=-1.0,
        vmax=1.0,
        cmap="icefire",
        method="complete",
        annot=annot,
        **kwargs,
    )

#corrplot(df_train, annot=None)

In [ ]:
#-------------------------
# Target Encoding
class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=5)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    # To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [ ]:
from category_encoders import CatBoostEncoder, MEstimateEncoder

def test_category_encoder():
    X, X_test = load_data_simple()
    encoder_me = CrossFoldEncoder(MEstimateEncoder, m=1)
    encoder_cbe = CrossFoldEncoder(CatBoostEncoder, a=2)

    X_encoded = encoder_cbe.fit_transform(X, y, cols=["MSSubClass"])
    X = X.join(encoder_cbe.fit_transform(X, y, cols=["MSSubClass"]))
    display(X['MSSubClass'])

In [ ]:
# label encoding for tree-based models
def label_encode_manual(df):
    '''tested: passing'''
    X = df.copy()
    for colname in X.select_dtypes(["category"]):
        X[colname] =  X[colname].cat.codes
        
    return X

# label encoding using sklearn module LabelEncoder
def label_encode_sk(df):
    '''tested: passing'''
    X = df.copy()
    categorical_columns = X.select_dtypes(["category"]).columns
    
    le = LabelEncoder()
    X[categorical_columns] = X[categorical_columns].apply(lambda x: le.fit_transform(x))
    return X

# One Hot encoding for linear regression based models, and svm
def one_hot_encode_pd(df):
    '''tested: passing'''
    X = df.copy()
    # for now we just use all categorical variables
    cols_for_one_hot = X.select_dtypes(["category"])
        
    X = pd.get_dummies(X, columns=cols_for_one_hot, prefix=str(cols_for_one_hot + "_"))
    return X
        
def one_hot_encode_sk(df):
    '''tested: passing'''
    X = df.copy()
    cols_for_one_hot = X.select_dtypes(["category"]).columns
    
    ohe = OneHotEncoder()
    X[cols_for_one_hot] = X[cols_for_one_hot].apply(lambda x: ohe.fit_transform(x))
    
    return X

def oh_encode_hosues2(df):
    pass


In [ ]:
def remove_outliers(df):
    outlier_mask = df['Outlier'] == 0
    df = df.loc[outlier_mask]
    return df

In [ ]:
def create_features_01(df, test_set=None, normalize=False, out_format='tree'):
    X = df.copy()
    y = X.pop("SalePrice")
    mi_scores = make_mi_scores(X, y)
    
    # combine data if necessary
    if test_set is not None:
        X_test = test_set.copy()
        X_test.pop("SalePrice")
        X = pd.concat([X, X_test])
        
    # Mutual Information
    X = drop_uninformative(X, mi_scores)
    
    # Transformations
    X = X.join(mathematical_transforms(X))
    X = X.join(interactions(X))
    X = X.join(porch_counts(X))
    #X = X.join(class_break_down(X))
    X = X.join(group_transforms(X))
    
    # Clustering
    X = X.join(cluster_labels(X, cluster_features, n_clusters=20))
    X = X.join(cluster_distance(X, cluster_features, n_clusters=20))
    
    # Principal Component Analysis
    X = X.join(pca_driven_features(X))
    X = X.join(pca_components(X, get_pca_feature_names()))
    
    X = X.join(indicate_outliers(X))
    
    X = label_encode_manual(X)
    
    # re-split data
    if test_set is not None:
        X_test = X.loc[test_set.index, :]
        X.drop(test_set.index, inplace=True)
        
    # Target Encoder options
    #encoder_me = CrossFoldEncoder(MEstimateEncoder, m=1)
    encoder_cbe = CrossFoldEncoder(CatBoostEncoder, a=2)
    
    X = X.join(encoder_cbe.fit_transform(X, y, cols=["MSSubClass"]))
    
        # Handle Optional Normalization
    if normalize: 
#         X = normalize_numerics(X)
        X = de_skew_yeo(X)
    
    # Handle Optional Test Set
    if test_set is not None: 
        X_test = X_test.join(encoder_cbe.transform(X_test))
    
    if test_set is not None:
        return X, X_test
    else:
        return X 

In [ ]:
def create_features_02(df, test_set=None, normalize=False, targ=True):
    X = df.copy()
    if targ:
        y = X.pop("SalePrice")
    mi_scores = make_mi_scores(X, y)
    
    # combine data if necessary
    if test_set is not None:
        X_test = test_set.copy()
        X_test.pop("SalePrice")
        X = pd.concat([X, X_test])
        
    # Mutual Information
    X = drop_uninformative(X, mi_scores)
    
    # Transformations1
    X = X.join(mathematical_transforms(X))
    X = X.join(interactions(X))
    X = X.join(porch_counts(X))
    #X = X.join(class_break_down(X))
    X = X.join(group_transforms(X))
    
    # Transformations2
    
    # Clustering
    X = X.join(cluster_labels(X, cluster_features, n_clusters=20))
    X = X.join(cluster_distance(X, cluster_features, n_clusters=20))
    
    # Principal Component Analysis
    X = X.join(pca_driven_features(X))
    X = X.join(pca_components(X, get_pca_feature_names()))
    
    # remove outliers handled prior to input
    #X = X.join(indicate_outliers(X))
    #X = remove_outliers(X) 
    
    # Handle Optional Normalization
    if normalize: 
        X = de_skew_yeo(X)
        
    # numerical encoding for categorical variables
    X = label_encode_manual(X)
    
    # re-split data
    if test_set is not None:
        X_test = X.loc[test_set.index, :]
        X.drop(test_set.index, inplace=True)
        
    # Target Encoder options
    #encoder_me = CrossFoldEncoder(MEstimateEncoder, m=1)
    encoder_cbe = CrossFoldEncoder(CatBoostEncoder, a=2)
    
    X = X.join(encoder_cbe.fit_transform(X, y, cols=["MSSubClass"]))
    
    # Handle Optional Test Set
    if test_set is not None: 
        X_test = X_test.join(encoder_cbe.transform(X_test))
    
    if test_set is not None:
        return X, X_test
    else:
        return X 

In [ ]:
import missingno as msno
def check_missing(X_train):
    df_miss = X_train.isna().sum().sort_values(ascending=False)

    cols_w_miss = df_miss[df_miss > 0].index
    print(X_train[cols_w_miss].info())

In [ ]:
def prepare_stacked_data(vers=1):
    # --------------------
    if vers == 1:
        print("loading data...")
        df_train, df_test = load_and_preproc_data()
        y_train = np.log(df_train["SalePrice"])
        df_train.drop(['Street'], axis=1)
        
        print("Engineering Features...")
        X_train, X_test = create_features_01(df_train, test_set=df_test, normalize= False)
        
        # remove outlier observations 
        X_train = X_train.loc[X_train['Outlier']==0]
        y_train =  y_train.loc[X_train.index]
        # remove outlier indicator column
        X_train = X_train.drop(['Outlier'], axis='columns')
        X_test = X_test.drop(['Outlier'], axis='columns')
        print("complete")
        return X_train, X_test, y_train, 
        
    if vers == 2:
        print("loading data...")
        df_train, df_test = load_and_preproc_data()
        y_train = np.log(df_train["SalePrice"])
        df_train.drop(['Street'], axis=1)
        
        df_train = df_train.join(indicate_outliers(df_train))
        df_train = remove_outliers(df_train)
        y_train = y_train.loc[df_train.index]
        
        print("Engineering Features...")
        X_train, X_test = create_features_02(df_train, test_set=df_test, normalize=True)
        
        print("complete")
        return X_train, X_test, y_train
    
    if vers == 3:
        df_train, df_test, df_train_y, df_test_y = load_and_preproc_data(type='tt_split')

        df_train = df_train.join(indicate_outliers(df_train))
        df_train = remove_outliers(df_train)
        df_train_y = df_train_y.loc[df_train.index]
        
        df_train = df_train.join(df_train_y)
        df_test = df_test.join(df_test_y)
        
        print("Engineering Features...")
        X_train, X_test = create_features_02(df_train, test_set=df_test, normalize=True)
        
        y_train = np.log(df_train_y)
        y_test = np.log(df_test_y)
        
        print("complete")
        return X_train, X_test, y_train, y_test

        # active

In [ ]:
#X_train_2, X_test_2, y_train_2, y_test_2 = prepare_stacked_data(vers=2)
X_n3, X_t3, y_n3, y_t3 = prepare_stacked_data(vers=3)

data_ref3 = [X_n3, X_t3, y_n3, y_t3]
def refresh_data():
    return data_ref3[0], data_ref3[1], data_ref3[2], data_ref3[3]

In [ ]:
X_train_2, X_test_2, y_train_2 = prepare_stacked_data(vers=2)
X_n2, X_t2, y_n2, y_t2 = train_test_split(X_train_2, y_train_2, test_size=0.2, random_state=42)

In [ ]:
# df_train, df_test = load_and_preproc_data()
# skew_cutoff=0.5
# X = df_train.copy()
# numeric_columns = X.select_dtypes(["number"]).columns
# skew_feats = X[numeric_columns].apply(lambda x: skew(x)).sort_values(ascending=False)

# too_skew = skew_feats[skew_feats > skew_cutoff].index


# # normalize each of the features with high skew with scipy boxcox 
#for s in too_skew:
#    X[s] = boxcox1p(X[s], boxcox_normmax(X[s] + 1))
# return X

In [ ]:

# test_col = 'PoolArea'
# # test_col = KitchenAbvGr 	

# X[test_col].dropna()
# print(f"pre cox: {X[test_col].skew()}")
# X[test_col] = np.log(X[test_col])
# # X[test_col] = boxcox(X[test_col], boxcox_normmax(X[test_col]))
# print(f" post cox: {X[test_col].skew()}")

In [ ]:
# df_train, df_test = load_and_preproc_data()

# skews = find_skew_cols(X_train_2)
# skews_o = find_skew_cols(df_train)

# test_skew_cols = [ col for col in skews_o.index.tolist() if col in skews.index.tolist()]
# pd.DataFrame([skews[test_skew_cols], skews_o[test_skew_cols]])['GrLivArea']



In [ ]:
# size_mask = X_train['GrLivArea'] > 4000
# cond_mask  = X_train.SaleCondition == "Partial"
# X_train.loc[cond_mask]["Outlier"]
# X_train["SaleCondition"].cat.codes

In [ ]:
# ---------------
# to delete - preserve for reference somewhere
# ---------------

def experiment_data_conversion():
    # save the datatypes
    in_dtypes = pd.DataFrame(X_train.dtypes)
    in_dtypes.reset_index(inplace=True)
    in_dtypes.columns = ['col', 'type']
    in_dtypes.to_csv('/kaggle/working/input_dtypes.csv', index=False)

    # Read back the data types
    input_types = pd.read_csv('/kaggle/working/input_dtypes.csv')
    input_dtypes = dict(zip(input_types.col, input_types.type))
    # other option
    # input_types.set_index('col').to_dict()['type']
    input_dtypes

In [ ]:
# wtf is going on here 
from sklearn.impute import SimpleImputer

def test_target_imputer():
    test_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_t_impute = test_imputer.fit_transform(X_test)

    return check_missing(X_t_impute)

---
---
---
---
# Chapter 2: Model Training
---
---
---
---

In [ ]:
# Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X, y, kf):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [ ]:
def get_stacked_scores(stk_model, X_dat, y_dat, method='orig'):
    stacked_models = [name for name in stk_model.named_estimators_.keys()]
    
    if method is 'orig':
        stacked_scores = [model.score(X_dat,y_dat) for model in stk_model.named_estimators_.values()]
        stk_scores = list(zip(stacked_models, stacked_scores))
        stk_scores.insert(0, ("stacked model", stk_model.score(X_dat, y_dat)))
    
    elif method is 'rmsle':
        stacked_scores = [score_dataset(X_dat,y_dat, model=model, enc=False, trans_targ=False) for model in stk_model.named_estimators_.values()]
        stk_scores = list(zip(stacked_models, stacked_scores))
        stack_rmsle = score_dataset(X_dat, y_dat, model=stk_model, enc=False, trans_targ=False)
        stk_scores.insert(0, ("stacked model", stack_rmsle))
    
    else:
        raise ValueError
    
    #print(stk_scores)
    return stk_scores

In [ ]:
# ------------------------------- #
# ------------------------------- # 
#     Stacked Model Section       #

#-------------------------------
# Preprocessing for stacked model
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

# Definining Cateogry Selectors
num_selector = make_column_selector(dtype_include=np.number)
cat_selector  = make_column_selector(dtype_include='object') #possibly change to 'object' if not using encode() utilit function

# tree preprocessor 
cat_tree_preprocessor = OrdinalEncoder()

#num_tree_preprocessor = SimpleImputer(strategy="mean", add_indicator=True)

tree_preprocessor = make_column_transformer(
     (cat_tree_preprocessor, cat_selector), #,(num_tree_preprocessor, num_selector) #todo: remove this imputation step handled in preprocess_data()
    remainder = 'passthrough'
    )

In [ ]:
# linear_preprocessor.fit_transform(X_train, y_train)

In [ ]:
# linear preprocessor 
cat_linear_selector = features_nom
cat_linear_preprocessor = OneHotEncoder(handle_unknown='ignore')
num_linear_preprocessor = make_pipeline(StandardScaler()) #,SimpleImputer(strategy='mean', add_indicator=True) 
                                                            #todo: this imputation step is handled in preprocess_data() -> simple_impute()                                     
linear_preprocessor = make_column_transformer(
                                            (num_linear_preprocessor, num_selector), 
                                            (cat_linear_preprocessor, cat_linear_selector) 
                                            )

In [ ]:


# todo  
# -------------------------
# partial dependence plots

# XX - optuna for stacked tree models
    #_histgb wouldn't work with optuna for some reason


# Shapley Value Analysis for Stacked Tree Models
# -- XG Boost Question

# Blended Model Version of Stacked Tree 

# Linear Models 
# remove colinearity for linear models 
# apply additional layer of simple math feature engineerings

# -----------------
# Case Studies
# - remodel old house add value 

# At fixed Price, Most valuable features

# Todo Complete
# XX - Remove Outliers 

In [ ]:
xgbst_pipeline = make_pipeline(
    tree_preprocessor, xgboost_stk
)

histgb_pipeline = make_pipeline(
    tree_preprocessor, HistGradientBoostingRegressor(random_state=0)
)

rf_pipeline = make_pipeline(
    tree_preprocessor, RandomForestRegressor(random_state=2)
)

In [ ]:
# ----------------------
# Linear Models
lasso_pipeline = make_pipeline(
    linear_preprocessor, LassoCV()
)

ridge_pipeline = make_pipeline(
    linear_preprocessor, RidgeCV()
)

svr_pipeline = Pipeline([
    ('prep',linear_preprocessor), 
    ('svr', SVR())
    ])

In [ ]:
X_nn = X_n3
X_tt = X_t3
y_nn = y_n3
y_tt = y_t3

lasso_pipeline.fit(X_nn, y_nn)
print(rmsle(y_nn, lasso_pipeline.predict(X_nn)))

ls_preds = lasso_pipeline.predict(X_tt)
lasso_pipeline.score(X_tt, y_tt)
rmsle(y_tt, ls_preds)

In [ ]:

ridge_pipeline.fit(X_n2, y_n2)
print(rmsle(y_n2, ridge_pipeline.predict(X_n2)))

rdg_preds = ridge_pipeline.predict(X_t2)
ridge_pipeline.score(X_t2, y_t2)
rmsle(y_t2, rdg_preds)

In [ ]:
X_n2, X_t2, y_n2, y_t2 = train_test_split(X_train_2, y_train_2, test_size=0.2, random_state=42)

# sv_params = {'svr__C' :[1,5,10],
#                  'svr__degree': [1,2,3,5,8],
#                  'svr__coef0': [0.01, 0.5, 10],
#                  'svr__gamma': ('auto', scale)
#             }

def run_gsearch_sv():
    sv_params={
                'svr__C': [0.1, 1, 100, 1000],
                'svr__epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
                'svr__gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
                },

    sv_grids = GridSearchCV(svr_pipeline, sv_params, cv=5, scoring='neg_mean_squared_error')
    sv_grids.fit(X_n2, y_n2)

    svr_pipeline_b = sv_grids.best_estimator_

    svr_pipeline_b.fit(X_n2, y_n2)
    svr_preds = svr_pipeline_b.predict(X_t2)
    svr_pipeline_b.score(X_t2, y_t2)
    print(rmsle(y_t2, svr_preds))
    return svr_pipeline_b
    
sv_results = run_gsearch_sv()

In [ ]:

X_n2, X_t2, y_n2, y_t2 = train_test_split(X_train_2, y_train_2, test_size=0.2, random_state=42)

sv_results.fit(X_n2, y_n2)
print(rmsle(y_n2, sv_results.predict(X_n2)))

sv_preds = sv_results.predict(X_t2)
sv_results.score(X_t2, y_t2)
rmsle(y_t2, sv_preds)

In [ ]:
rmsle(y_n2, svr_pipeline_b.predict(X_n2))

In [ ]:
lin_prep_pipeline = Pipeline([('prep', linear_preprocessor)])

In [ ]:
lasso_dat = pd.Concat(preds, y_t2)
lassoo_dat

In [ ]:
shap_force(20, lasso_pipeline)

In [ ]:
X_train.info()

In [ ]:
rand_seed = 2

In [ ]:
# X_train = create_features(df_train)
# y_train = df_train.loc[:, "SalePrice"]

def run_xgb_optuna():
    
    def objective(trial):
        xgb_params = dict(
            max_depth=trial.suggest_int("max_depth", 2, 10),
            learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
            n_estimators=trial.suggest_int("n_estimators", 1000, 8000),
            min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
            colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
            subsample=trial.suggest_float("subsample", 0.2, 1.0),
            reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
            reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
        )
        xgb = XGBRegressor(**xgb_params, random_state=rand_seed)
        return score_dataset(X_train, y_train, xgb, enc=False, trans_targ=False)
    
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10)
    
    return study.best_params

# xgb_param_stk = run_xgb_optuna()

In [ ]:
xgb_params_opt = {
    'max_depth': 4,
     'learning_rate': 0.05354363798458815,
     'n_estimators': 5156,
     'min_child_weight': 6,
     'colsample_bytree': 0.334883002631706,
     'subsample': 0.9095426160373565,
     'reg_alpha': 0.009835234945007399,
     'reg_lambda': 0.017678554214634153}

In [ ]:
def run_histgb_optuna():
    
    # creating index mask for categorical features argument
    # creating index mask for categorical features argument
    # would I need to check the engineered features as well? 
    updt_cat_feats = [feat  for feat in features_cat if feat in X_train.columns]
    cat_feature_indxs = [X_train.columns.get_loc(feat) for feat in updt_cat_feats]

    def objective(trial):
        histgb_params = dict(
            learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e2, log=True),
            max_leaf_nodes = trial.suggest_int("max_leaf_nodes",1,50),
            # max_depth = trial.suggest_int( default=None),
            # max_iter = trial.suggest_int("max_iter", defualt=100),
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 10, 30), # default=20; 
            l2_regularization = trial.suggest_float("l2_regularization",1e-4,1e2,log=True),
            max_bins = trial.suggest_float("max_bins",100,255), #max bins for non-missing values, must be no larger than 255
            #categorical_features = cat_feature_indxs # new in version 0.24
        )
        
        hgb = HistGradientBoostingRegressor(**histgb_params, random_state=rand_seed)
        return score_dataset(X_train, y_train, hgb, enc=False, trans_targ=False)
    
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10)
    
    return study.best_parameters

# histgb_param_stk = run_histgb_optuna()

In [ ]:
def run_rf_optuna():
    
    def objective(trial):

        rf_params = dict(
            n_estimators = trial.suggest_int("n_estimators", 50, 200),
            #max_depth defaul=None
            min_samples_split = trial.suggest_int("min_samples_split", 2,6), # default = 2
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10), # defualt = 1
            #min_weight_fraction_leaf : float; default = 0.0
            #max_features = # default = 'auto'
            #max_leaf_nodes = # default = None
            #min_impurity_decrease # float, defualt= 0.0
        )

        rf = RandomForestRegressor(**rf_params, random_state=rand_seed)
        return score_dataset(X_train, y_train, rf, enc=False, trans_targ=False)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10)

    return study.best_params

# rf_param_stk = run_rf_optuna()

In [ ]:
rf_params_opt = {'n_estimators': 132, 'min_samples_split': 4, 'min_samples_leaf': 2}

In [ ]:
rand_seed = 2

xgb_params_opt = {
    'max_depth': 4,
     'learning_rate': 0.05354363798458815,
     'n_estimators': 5156,
     'min_child_weight': 6,
     'colsample_bytree': 0.334883002631706,
     'subsample': 0.9095426160373565,
     'reg_alpha': 0.009835234945007399,
     'reg_lambda': 0.017678554214634153}

rf_params_opt = {'n_estimators': 132, 'min_samples_split': 4, 'min_samples_leaf': 2}

# make Stacked Model
def train_stacked_model_tree():
    # --------------------
    stack_estimators = {
        ("XGBoost", XGBRegressor(**xgb_params_opt, random_state=rand_seed)),
        #("HistGB", HistGradientBoostingRegressor(random_state=rand_seed)),
        #("Random Forest", RandomForestRegressor(**rf_params_opt, random_state=rand_seed)),  
        ("lasso", lasso_pipeline),
        ("ridge", ridge_pipeline),
        ("svr", sv_results)
    }

    #final_reg = XGBRegressor(random_state=2)
    final_reg = RidgeCV()

    print('Creating Initial Model...')
    stacked_model = StackingRegressor(estimators=stack_estimators, final_estimator=final_reg)
    print('Training Underway...')
    stacked_model.fit(X_n2, y_n2)
    print(rmsle(y_t2, stacked_model.predict(X_t2)))
    return stacked_model

#    End Stacked Model Section    #
# ------------------------------- #
# ------------------------------- # 

In [ ]:
final_stack_model_tree = train_stacked_model_tree()

In [ ]:
rmsle(y_t2, final_stack_model_tree.predict(X_t2))

In [ ]:
get_stacked_scores(final_stack_model_tree, X_train, y_train, method='rmsle')

In [ ]:
# score_dataset(X_train, y_train, model=rf_sub_est ,enc=False, trans_targ=False)

In [ ]:
X_n, X_t, y_n, y_t = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Setup cross validation folds
kf = KFold(n_splits=12, random_state=0, shuffle=True)

In [ ]:
rf_bl = RandomForestRegressor(**rf_params_opt, random_state=rand_seed)
# ----------------
#rf_bl.fit(X_train, y_train)
# print(score_dataset(X_train, y_train, rf_bl, enc=False, trans_targ=False))
# ----------------
# rf_bl.fit(X_n, y_n)
# print(rf_bl.score(X_t, y_t))
# print(rmsle(y_t, rf_bl.predict(X_t)))

In [ ]:
rf_bl_preds = rf_bl.predict(X_train).tolist()

In [ ]:
xgb_bl = XGBRegressor(**xgb_params_opt, random_state=rand_seed)
# ----------------
# xgb_bl.fit(X_train, y_train)
# print(score_dataset(X_train, y_train, xgb_bl, enc=False, trans_targ=False))
# ----------------
# xgb_bl.fit(X_n, y_n)
# print(xgb_bl.score(X_t, y_t))
# print(rmsle(y_t, xgb_bl.predict(X_t)))

In [ ]:
xgb_bl_preds = xgb_bl.predict(X_train).tolist()

In [ ]:
hgb_bl = HistGradientBoostingRegressor(random_state=rand_seed)
# ----------------
# hgb_bl.fit(X_train, y_train)
# print(score_dataset(X_train, y_train, hgb_bl, enc=False, trans_targ=False))
# ----------------
hgb_bl.fit(X_n, y_n)
print(hgb_bl.score(X_t, y_t))
print(rmsle(y_t, hgb_bl.predict(X_t)))

In [ ]:
# score_dataset(X_train, y_train, enc=False, trans_targ=False)

In [ ]:
#------------------------
# Blended Model
#------------------------
def train_blended_nt():
    #X_n, X_t, y_n, y_t = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    models_dict = {'xgb': {'model': xgb_bl, 'weight': 0.7},
                  #'rforest': {'model': rf_bl, 'weight': 0.3},
                   'lasso_pip': {'model': lasso_pipeline, 'weight': 0.4},
                   'ridge_pip': {'model': ridge_pipeline, 'weight': 0.2},
                   'svr_pip': {'model': sv_results, 'weight': 0.4}
                  #'hgb': {'model': hgb_bl, 'weight': 0.25}
                  #'stacked': {'model': stacked_model, 'weight': 0.25}}
                  }

    reg_models = [(key, value['model']) for key, value in models_dict.items()]
    vote_weights = [value['weight'] for value in models_dict.values()]

    vote_reg = VotingRegressor(estimators=reg_models)
    # --------------
    # vote_reg.fit(X_train, y_train)
    # print(score_dataset(X_train, y_train, vote_reg, enc=False, trans_targ=False))
    
    # --------------
    vote_reg.fit(X_n2, y_n2)
    print(rmsle(y_t2, vote_reg.predict(X_t2)))
    return vote_reg

vote_reg_p = train_blended_nt()

In [ ]:
rmsle(y_n2, vote_reg_p.predict(X_n2))

In [ ]:
vote_reg_p.named_estimators_['lasso_pip'].named_steps['lassocv'].get_params()

In [ ]:
vote_reg_p.named_estimators_

In [ ]:
vote_reg_p.feature_importance_

___ 
___
Linear Ensemble Models

In [ ]:
# -----------------
# Stacked Linear Model 
# ----------------

def train_linear_stacked_model():
    stack_estimators_lm = [
        #("lasso_stk", lasso_pipeline),
        ("ridge_stk", ridge_pipeline),
        ("svr_stk", svr_pipeline)
    ]
    
    final_reg_lm = RidgeCV()
    
    print('Creating Initial Model...')
    stacked_model_lm = StackingRegressor(estimators=stack_estimators_lm, final_estimator=final_reg_lm)
    print('Training Underway...')
    stacked_model_lm.fit(X_train, y_train)
    #stacked_model.score(X_test, Y_test)
    print("training complete")
    return stacked_model_lm

In [ ]:
# final_stacked_linear_model = train_linear_stacked_model()

In [ ]:
# final_stacked_linear_model.score(X_train, y_train)
# get_stacked_scores(final_stacked_linear_model, X_train, y_train, method='rmsle')

In [ ]:
# score_dataset(X_train, y_train, model=final_stacked_linear_model, enc=False, trans_targ=False)

In [ ]:
# get_stacked_scores(final_stacked_linear_model, X_train, y_train)

In [ ]:
# -----------------
# Combined Stacked Model 
# ----------------

def train_mixed_stacked_model():
    
    stack_estimators_mx = [
        ("ridge_stk", svr_pipeline), 
        ("XGBoost", XGBRegressor(random_state=0)),
        ("HistGB", HistGradientBoostingRegressor(random_state=0)),
        ("Random Forest", RandomForestRegressor(random_state=2)), 
    ]

    #final_reg_mx = RandomForestRegressor(random_state=0)
    final_reg_mx = XGBRegressor(random_state=2)    
    #final_reg_mx = RidgeCV()
    
    print('Creating Initial Model...')
    stacked_model_mx = StackingRegressor(estimators=stack_estimators_mx, 
                                         final_estimator=final_reg_mx, 
                                        passthrough=True)
    
    print('Training Underway...')
    stacked_model_mx.fit(X_train, y_train)
    #stacked_model.score(X_test, Y_test)
    print("training complete")
    return stacked_model_mx
    

In [ ]:
final_mx_stacked_model = train_mixed_stacked_model()
get_stacked_scores(final_mx_stacked_model, X_train, y_train, method='rmsle')

In [ ]:
# final_mx_stacked_model.named_estimators_

In [ ]:
# mx_stk_model = final_mx_stacked_model
# histgb_pred = mx_stk_model.named_estimators_['HistGB'].predict(X_train)

In [ ]:
# hist_rval = np.exp(histgb_pred)
# y_train_rval = np.exp(y_train)
# hist_rval.shape
# histgb_pred.shape
# y_train.shape

In [ ]:
# comp_reals = pd.DataFrame(hist_rval, y_train_rval).reset_index()
# comp_reals.columns = ['hist_rval', 'y_train_rval']
# comp_reals['diff'] = comp_reals['hist_rval'] - comp_reals.y_train_rval
# abs(comp_reals['diff']).max()

In [ ]:
# import seaborn as sns
# sns.scatterplot(data=comp_reals, x=comp_reals.index, y='diff')

--- 
---
# Blended Model Interpretability
---
---

In [ ]:
# reference on getting explainer for a model

# explainer = shap.TreeExplainer(my_model)


In [ ]:
# -----------------
# Investigate Shap Values for single prediction

def shap_force(row_to_show, explainer):
    val_X = X_train
    small_val_X = X_train.iloc[:150]
    data_investigate = val_X.iloc[row_to_show]
    
    shap_values = explainer.shap_values(data_investigate)
    # display shap value graphic
    shap.initjs()
    return shap.force_plot(explainer.expected_value, shap_values, data_investigate)

In [ ]:
# --------------------
def shap_summary(explainer):
    val_X = X_train
    small_val_X = X_train.iloc[:150]
    shap_values = explainer.shap_values(val_X)
    shap.summary_plot(shap_values, val_X)


In [ ]:
def gen_pdp_plot(my_model, feat_name, model_feats):
    """
    model_feats : list 
    feat_name : str
    """
    pdp_dist = pdp.pdp_isolate(model=my_model, dataset=X_train, model_features=model_feats, feature=feat_name)
    pdp.pdp_plot(pdp_dist, feat_name)
    plt.show()

In [ ]:
def gen_perm_import(my_model, feat_names):
    """
    feat_names : list
    """
    perm = PermutationImportance(my_model).fit(X_train, y_train)

    # show the weights for the permutation importance 
    eli5.show_weights(perm, feature_names = feat_names)

In [ ]:
vote_reg_perm = PermutationImportance(vote_reg_p).fit(X_n2, y_n2)
eli5.show_weights(vote_reg_perm, feature_names = X_train.columns.tolist())

In [ ]:
vote_reg_perm

In [ ]:
gen_pdp_plot(vote_reg_p, 'GarageArea', X_train.columns.tolist())

In [ ]:
gen_pdp_plot(vote_reg_p, 'GarageCars', X_train.columns.tolist())

In [ ]:
# what is spaciousness
gen_pdp_plot(vote_reg_p, 'Spaciousness', X_train.columns.tolist())

In [ ]:
gen_pdp_plot(vote_reg_p, 'YearRemodAdd', X_train.columns.tolist())

In [ ]:
gen_pdp_plot(vote_reg_p, 'Feature1', X_train.columns.tolist())

In [ ]:
gen_pdp_plot(vote_reg_p, 'Fireplaces', X_train.columns.tolist())

In [ ]:
gen_pdp_plot(vote_reg_p, 'YearBuilt', X_train.columns.tolist())

In [ ]:
gen_pdp_plot(hgb_bl, 'YearBuilt', X_train.columns.tolist())

In [ ]:
gen_pdp_plot(vote_reg_p, 'YearBuilt', X_train.columns.tolist())

In [ ]:
rf_shap_values = rf_bl_exp.shap_values(val_X)

In [ ]:
# ----------------------
# Interpreting Single Models
# ----------------------
val_X = X_train
small_val_X = X_train.iloc[:150]

In [ ]:
lasso_pipeline.fit(X_n2, y_n2)


In [ ]:
rf_bl_exp = shap.TreeExplainer(rf_bl)
hgb_bl_exp = shap.TreeExplainer(hgb_bl)

In [ ]:
#v_reg_exp = shap.TreeExplainer(vote_reg)
lasso_bl_exp = shap.Explainer(lasso_pipeline)

In [ ]:
shap_summary(rf_bl_exp)

In [ ]:
shap_summary(hgb_bl_exp)

In [ ]:
shap_force(20, )

In [ ]:
shap_force(20, rf_bl_exp)

In [ ]:
shap_force(20, xgb_bl_exp)

In [ ]:
# problem solving the XGB Shapley Values
# xgb_y_preds = xgb_bl.predict(X_train)
xgb_bl_exp = shap.TreeExplainer(xgb_bl)

In [ ]:
#shap_force(10, xgb_bl_exp)
row_to_show = 10
val_X = X_train
small_val_X = X_train.iloc[:150]

data_investigate = val_X.iloc[row_to_show]

shap_values = xgb_bl_exp.shap_values(data_investigate, check_additivity=False)

# display shap value graphic
# shap.initjs()
# shap.force_plot(explainer.expected_value, shap_values[0], data_investigate)

In [ ]:
shap.summary_plot(shap_values, small_val_X, plot_type='bar')

In [ ]:
feature_of_interest = 'Feature1'
interaction_feature = 'MedNgbdArea'
shap.dependence_plot(feature_of_interest, shap_values, small_val_X, interaction_index=interaction_feature)

In [ ]:
feature_of_interest = 'LotArea'
interaction_feature = 'GrLivArea'
shap.dependence_plot(feature_of_interest, shap_values, small_val_X, interaction_index=interaction_feature)

In [ ]:
data_for_prediction = val_X.iloc[0,:]  # use 1 row of data here. Could use multiple rows if desired

# Create object that can calculate shap values
explainer = shap.TreeExplainer(my_model)
shap_values = explainer.shap_values(data_for_prediction)
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0], data_for_prediction)
# ------------

In [ ]:
# Summarizing Shapley Values

In [ ]:
#data_for_prediction
data_investigate

In [ ]:
explainer.expected_value
data_investigate.shape

---
--- 

## Individual and Blended Models
---
---

In [ ]:
#---------------------------
# My Data Pipeline 
# ---------------------------

X_train, X_test, y_train = prepare_stacked_data()
data_reference = (X_train, X_test, y_train)

In [ ]:
# Setup cross validation folds
kf = KFold(n_splits=12, random_state=0, shuffle=True)

In [ ]:
# ------------------
# Ridge Model Fitting
# ------------------

ridge_rsearch = RandomizedSearchCV(ridge_model, param_distributions=ridge_param_dist, n_iter=n_iter_search)
ridge_rsearch.fit(X_train, y_train)
ridge_b_model = rf_gsearch.best_estimator_
ridge_score = ridge_rsearch.best_score_

In [ ]:
# ------------------
# SVR Model Fitting
# ------------------

svr_rsearch = RandomizedSearchCV(svr_model, param_distributions=svr_param_dist, n_iter=n_iter_search)
svr_rsearch.fit(X_train, y_train)
svr_b_model = svr_rsearch.best_estimator
svr_score = svr_rsearch.best_score_

---- 
----
# Old Code
----
----

In [ ]:
# -------------
# Aside: note
# -------------

# scipy uniform understanding 
# import matplotlib.pyplot as plt
# r = uniform.rvs(loc=1, scale=4, size=1000)

# fig, ax = plt.subplots(1, 1)
# ax.hist(r, density=True, histtype='stepfilled', alpha=0.2)
# ax.legend(loc='best', frameon=False)
# plt.show()

In [ ]:
#---------------------------------------
# Hyper Parameter Tuning for Individual Models
# --------------------------------------
from scipy.stats import uniform 

# XGBoost Model Setup
xgb_model = XGBRegressor(
                   objective='reg:squarederror',
                   nthread=-1,
                   scale_pos_weight=1,
                   seed=11,
                   random_state=2
                   )

xgb_param_dist = dict(learning_rate= uniform(0.005, 0.1),
                   n_estimators=1000*np.arange(4,14,2),
                   max_depth=np.arange(4,6,1),
                   min_child_weight=np.arange(0, 1, 0.1),
                   gamma=uniform(0.3,0.8),
                   subsample= uniform(0.4,1),
                   colsample_bytree= uniform(0.6,1),
                   reg_alpha=uniform(0.00001, 0.0001)
                   )


xgb_param_grid = dict(learning_rate= np.arange(0.005, 0.1, 0.005),
                   n_estimators=1000*np.arange(4,14,2),
                   max_depth=np.arange(4,6,1),
                   min_child_weight=np.arange(0, 1, 0.1),
                   gamma=np.arange(0.3,0.8,0.1),
                   subsample= np.arange(0.6, 1.1, 0.1),
                   colsample_bytree= np.arange(0.6, 1.1, 0.1),
                   reg_alpha=np.linspace(0.00001, 0.0001)
                   )

# Random Forest Model Setup
rf_model = RandomForestRegressor(max_features=None,
                    oob_score=True,
                    random_state=42
                    )

rf_param_grid = dict(n_estimators=np.arange(800,1200,200),
                    max_depth=np.arange(7,20),
                    min_samples_split=np.arange(3,10),
                    min_samples_leaf=np.arange(3,7,1),
                    )

# Ridge Model Setup
ridge_model = Ridge(solver='lsqr')

ridge_param_dist = dict(alphas = uniform(1e-15, 100))

# SVR Model Setup
svr_model = SVR()

svr_param_dist = dict(C=uniform(0.01, 10), 
                      epsilon= uniform(0.001,0.5)
                     )

# Todo: Decide if we will use these
# lasso model setup
# elastic model setup

In [ ]:
# ----------------------------
# Optuna enet

def objective(trial):
    enet_params = dict(
        alpha=trial.suggest_float("alpha", 1e-3, 10.0),
        l1_ratio=trial.suggest_float("l1_ratio", 0.01, 1.0),
    )
    
    enet = ElasticNet(**enet_params)
    return score_dataset(X_train, y_train, ridge_model, enc=False, trans_targ=False)

enet_study = optuna.create_study(direction="minimize")
enet_study.optimize(objective, n_trials=3)
enet_params_tuna = ridge_study.best_params


In [ ]:
# ------------------
# XGB Model Fitting
# ------------------

n_iter_search = 15
xgb_rsearch = RandomizedSearchCV(xgb_model, param_distributions=xgb_param_dist, n_iter=n_iter_search)
xgb_rsearch.fit(X_train, y_train)
xgb_b_model = xgb_rsearch.best_estimator_
xgb_score = xgb_rsearch.best_score_
print(xgb_score)

In [ ]:
print(cv_rmse(xgb_b_model, X_train, y_train))

In [ ]:
# ------------------
# Random Forest Model Fitting
# ------------------

rf_gsearch = RandomizedSearchCV(rf_model, param_distributions=rf_param_dist, n_iter=n_iter_search)
rf_gsearch.fit(X_train, y_train)
rf_b_model = rf_gsearch.best_estimator_
rf_score = rf_gsearch.best_score_

In [ ]:
score_dataset(X_train, y_train, rf_b_model, enc=False, trans_targ=False)

In [ ]:
#------------------------
# Blended Model
#------------------------

# sklearn version "VotingRegressor"
from sklearn.ensemble import VotingRegressor

reg_models = {'xgb': {'model': xgb_b_model, 'weight': 0.2},
              #'lasso': {'model': lasso_b_model, 'weight': 0.2},
              'rforest': {'model': rf_b_model, 'weight': 0.2},
              'ridge': {'model': ridge_b_model, 'weight': 0.2},
              'svr'  : {'model': svr_b_model, 'weight': 0.2},
              'stacked': {'model': stacked_model, 'weight': 0.2}}

reg_models = [(key, value['model']) for key, value in models_dict.items()]
vote_weights = [value['weight'] for value in models_dict.values()]

vote_reg = VotingRegressor(estimators=reg_models, weights=vote_weights)

---
---
---
# Sub Chapter: Linear Model Experiment
---
---
---

In [ ]:
# ------------------
# Ridge Model Fitting
# ------------------

# Setup cross validation folds
kf = KFold(n_splits=12, random_state=0, shuffle=True)

ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
ridge_solo_pipe = make_pipeline(linear_preprocessor, RidgeCV(alphas=ridge_alphas, cv=kf))
ridge_solo_pipe.fit(X_train, y_train)
ridge_solo_pipe.score(X_train, y_train)

In [ ]:
l1_ratios = [.1, .5, .7, .9, .95, .99, 1]
elastic_solo_pipe = make_pipeline(linear_preprocessor, ElasticNetCV(l1_ratio=l1_ratios ,cv=kf))
elastic_solo_pipe.fit(X_train, y_train)
elastic_solo_pipe.score(X_train, y_train)

---
# end: linear sub Chapter

---
---
---

In [ ]:
# ------------------
# Ridge Model Fitting
# ------------------
ridge_rsearch = RandomizedSearchCV(ridge_model, param_distributions=ridge_param_dist, n_iter=n_iter_search)

ridge_rsearch.fit(X_train, y_train)
ridge_b_model = rf_gsearch.best_estimator_
ridge_score = ridge_rsearch.best_score_

In [ ]:
# Light Gradient Boosting Regressor
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=7000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

# XGBoost Regressor
xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:linear',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

# Ridge Regressor
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

# Support Vector Regressor
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=6000,
                                learning_rate=0.01,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)  

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=1200,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

# Stack up all the models above, optimized using xgboost
stack_gen = StackingCVRegressor(regressors=(xgboost, lightgbm, svr, ridge, gbr, rf),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [ ]:
# -----------------------
# Fit the final 0.3 models
# -----------------------

In [ ]:
print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(train_labels))

In [ ]:
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X_train, y_train)

In [ ]:
print('xgboost')
xgb_model_full_data = xgboost.fit(X_train, y_train)

In [ ]:
print('Svr')
svr_model_full_data = svr.fit(X_train, y_train)

In [ ]:
print('Ridge')
ridge_model_full_data = ridge.fit(X_train, y_train)

In [ ]:
print('RandomForest')
rf_model_full_data = rf.fit(X_train, y_train)

In [ ]:
print('GradientBoosting')
gbr_model_full_data = gbr.fit(X_train, y_train)

In [ ]:
# -----------------------
# Blended Model 0.3
# -----------------------

# sklearn version "VotingRegressor"
from sklearn.ensemble import VotingRegressor

reg_models = {'xgb': {'model': xgb_model, 'weight', 0.2},
              'lasso': {'model': lasso_model, 'weight': 0.2},
              'ridge': {'model': ridge_model, 'weight': 0.2},
              'svr'  : {'model': svr_model, 'weight': 0.2},
              'stacked': {'model': stacked_model, 'weight': 0.2}}

reg_models = [(key, value['model']) for key, value in models_dict.items()]
vote_weights = [value['weight'] for value in models_dict.values()]

vote_reg = VotingRegressor(estimators=reg_models, weights=vote_weights)

In [ ]:
# Get final precitions from the blended model
blended_score = rmsle(y_train, VotingRegressor.fit(X_train))
scores['blended'] = (blended_score, 0)
print('RMSLE score on train data:')
print(blended_score)

---
---
---
# Model Interpretation
---
---
---